<a href="https://colab.research.google.com/github/openrankprotocol/interest-and-credit-notebooks/blob/main/OpenRank%20%2B%20GitHub%20I%26C%20for%20Retrofunding%20R5%20Voter%20Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openrank-sdk

from google.colab import auth
from google.cloud import bigquery
from openrank_sdk import EigenTrust
import pandas

auth.authenticate_user()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


# Create I&C Local Trust

In [11]:
%%bigquery dataframe --project ek-oso-test

WITH
  -- see https://gov.optimism.io/t/8393
  timebox AS (
    SELECT
      TIMESTAMP '1980-01-01 00:00:00 UTC' AS start,
      TIMESTAMP '2024-06-27 00:00:00 UTC' AS `end`
  ),
  bot_users AS (SELECT * FROM UNNEST([
    'safe-infra'
  ]) AS username),
  repo_event_types AS (
    SELECT DISTINCT event_type
    FROM `opensource-observer.oso.int_events`
    WHERE from_artifact_type = 'GIT_USER' AND to_artifact_type = 'REPOSITORY'
  ),
  -- user-to-repo trust components, signals interest/trust
  trust_event_weights AS (
    SELECT * FROM UNNEST(ARRAY<STRUCT<event_type STRING, weight INT64>>[
      ('FORKED', 1),
      ('STARRED', 5),
      ('ISSUE_OPENED', 10),
      ('PULL_REQUEST_OPENED', 20),
      -- added EK 8/3
      ('PULL_REQUEST_MERGED', 10),
      ('COMMIT_CODE', 5)
    ])
  ),
  -- repo-to-user trust components
  credit_event_weights AS (
    SELECT * FROM UNNEST(ARRAY<STRUCT<event_type STRING, weight INT64>>[
      ('PULL_REQUEST_OPENED', 5), -- TODO(ek): gameable, only count merged ones
      ('PULL_REQUEST_MERGED', 1), -- to the one who merged PR, not the author
      ('COMMIT_CODE', 3)
    ])
  ),
  repo_events AS (
    SELECT
      `time`,
      -- events' importance exponentially decay, with the half life of 1 year
      POW(0.5, TIMESTAMP_DIFF(timebox.`end`, time, SECOND) / 31536000) AS age_factor,
      event_type,
      from_artifact_name AS `from`,
      to_artifact_namespace || '/' || to_artifact_name AS `to`,
      amount
    FROM `opensource-observer.oso.int_events`
    JOIN timebox ON `opensource-observer.oso.int_events`.time BETWEEN timebox.start AND timebox.`end`
    JOIN repo_event_types USING (event_type)
    LEFT OUTER JOIN bot_users ON from_artifact_name = bot_users.username
    WHERE bot_users.username IS NULL
  )
SELECT
  `from` AS i,
  `to` AS j,
  SUM(amount * age_factor * weight) AS v
FROM repo_events
JOIN trust_event_weights USING (event_type)
GROUP BY i, j
HAVING i NOT LIKE '%[bot]' AND i NOT LIKE '%-bot'
UNION ALL
SELECT
  `to` AS i,
  `from` AS j,
  SUM(amount * age_factor * weight) AS v
FROM repo_events
JOIN credit_event_weights USING (event_type)
GROUP BY i, j
HAVING j NOT LIKE '%[bot]' AND j NOT LIKE '%-bot'
ORDER BY v DESC


Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
dataframe

,i,j,v
0,samczsun,metamask/eth-phishing-detect,190884.215234
1,code423n4,code-423n4/code423n4.com,170533.441437
2,metamask/eth-phishing-detect,samczsun,114390.660669
3,matrixbot,element-hq/synapse,113642.377667
4,coreycaplan3,dolomite-exchange/liquidity-mining-data,76907.536424
...,...,...,...
3300610,magiclu,clowwindy/chinadns-c,0.001389
3300611,yhfolive,internetarchive/heritrix3,0.001389
3300612,hynnet,shadowsocks/shadowsocks-android,0.001388
3300613,harryert,clowwindy/chinadns,0.001388


In [13]:
localtrust = list(dataframe.to_dict("records"))

In [17]:
pretrust = [
    dict(i='testinprod-io/op-erigon', v=2),
    dict(i='a16z/magi', v=2),
    dict(i='ethereum-optimism/optimism', v=2),
    dict(i='ethereum-optimism/op-geth', v=2),
    dict(i='ethereum-optimism/asterisc', v=2),
    dict(i='ethereum-optimism/kona', v=2),
    dict(i='ethereum-optimism/superchain-ops', v=1),
    dict(i='ethereum-optimism/op-analytics', v=1),
    dict(i='ethereum-optimism/design-docs', v=2),
    dict(i='ethereum-optimism/specs', v=2),
]
a = EigenTrust(host_url='https://ek-go-eigentrust.k3l.io')
scores = a.run_eigentrust(localtrust, pretrust, alpha=0.2, check_freq=2, scale='raw')
scores

ERROR:root:Server returned HTTP 504: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>

ERROR:root:error while sending a request to go-eigentrust
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/openrank_sdk/eigentrust_client.py", line 522, in _send_go_eigentrust_req
    raise RuntimeError(msg)
RuntimeError: Server returned HTTP 504: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>



RuntimeError: Server returned HTTP 504: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>
